In [1]:
from PIL import Image
import numpy as np
import time

In [2]:
# Source of perlin noise https://stackoverflow.com/questions/42147776/producing-2d-perlin-noise-with-numpy

def perlin(x,y,): # seed=0):
    # permutation table
    # np.random.seed(seed)
    p = np.arange(256,dtype=int)
    np.random.shuffle(p)
    p = np.stack([p,p]).flatten()
    # coordinates of the top-left
    xi = x.astype(int)
    yi = y.astype(int)
    # internal coordinates
    xf = x - xi
    yf = y - yi
    # fade factors
    u = fade(xf)
    v = fade(yf)
    # noise components
    n00 = gradient(p[p[xi]+yi],xf,yf)
    n01 = gradient(p[p[xi]+yi+1],xf,yf-1)
    n11 = gradient(p[p[xi+1]+yi+1],xf-1,yf-1)
    n10 = gradient(p[p[xi+1]+yi],xf-1,yf)
    # combine noises
    x1 = lerp(n00,n10,u)
    x2 = lerp(n01,n11,u) # FIX1: I was using n10 instead of n01
    return lerp(x1,x2,v) # FIX2: I also had to reverse x1 and x2 here

# source of 3d perlin noise https://pvigier.github.io/2018/11/02/3d-perlin-noise-numpy.html
def perlin_3d(shape, res):
    def f(t):
        return 6*t**5 - 15*t**4 + 10*t**3

    delta = (res[0] / shape[0], res[1] / shape[1], res[2] / shape[2])
    d = (shape[0] // res[0], shape[1] // res[1], shape[2] // res[2])
    grid = np.mgrid[0:res[0]:delta[0],0:res[1]:delta[1],0:res[2]:delta[2]]
    grid = grid.transpose(1, 2, 3, 0) % 1
    # Gradients
    theta = 2*np.pi*np.random.rand(res[0]+1, res[1]+1, res[2]+1)
    phi = 2*np.pi*np.random.rand(res[0]+1, res[1]+1, res[2]+1)
    gradients = np.stack((np.sin(phi)*np.cos(theta), np.sin(phi)*np.sin(theta), np.cos(phi)), axis=3)
    gradients[-1] = gradients[0]
    g000 = gradients[0:-1,0:-1,0:-1].repeat(d[0], 0).repeat(d[1], 1).repeat(d[2], 2)
    g100 = gradients[1:  ,0:-1,0:-1].repeat(d[0], 0).repeat(d[1], 1).repeat(d[2], 2)
    g010 = gradients[0:-1,1:  ,0:-1].repeat(d[0], 0).repeat(d[1], 1).repeat(d[2], 2)
    g110 = gradients[1:  ,1:  ,0:-1].repeat(d[0], 0).repeat(d[1], 1).repeat(d[2], 2)
    g001 = gradients[0:-1,0:-1,1:  ].repeat(d[0], 0).repeat(d[1], 1).repeat(d[2], 2)
    g101 = gradients[1:  ,0:-1,1:  ].repeat(d[0], 0).repeat(d[1], 1).repeat(d[2], 2)
    g011 = gradients[0:-1,1:  ,1:  ].repeat(d[0], 0).repeat(d[1], 1).repeat(d[2], 2)
    g111 = gradients[1:  ,1:  ,1:  ].repeat(d[0], 0).repeat(d[1], 1).repeat(d[2], 2)
    # Ramps
    n000 = np.sum(np.stack((grid[:,:,:,0]  , grid[:,:,:,1]  , grid[:,:,:,2]  ), axis=3) * g000, 3)
    n100 = np.sum(np.stack((grid[:,:,:,0]-1, grid[:,:,:,1]  , grid[:,:,:,2]  ), axis=3) * g100, 3)
    n010 = np.sum(np.stack((grid[:,:,:,0]  , grid[:,:,:,1]-1, grid[:,:,:,2]  ), axis=3) * g010, 3)
    n110 = np.sum(np.stack((grid[:,:,:,0]-1, grid[:,:,:,1]-1, grid[:,:,:,2]  ), axis=3) * g110, 3)
    n001 = np.sum(np.stack((grid[:,:,:,0]  , grid[:,:,:,1]  , grid[:,:,:,2]-1), axis=3) * g001, 3)
    n101 = np.sum(np.stack((grid[:,:,:,0]-1, grid[:,:,:,1]  , grid[:,:,:,2]-1), axis=3) * g101, 3)
    n011 = np.sum(np.stack((grid[:,:,:,0]  , grid[:,:,:,1]-1, grid[:,:,:,2]-1), axis=3) * g011, 3)
    n111 = np.sum(np.stack((grid[:,:,:,0]-1, grid[:,:,:,1]-1, grid[:,:,:,2]-1), axis=3) * g111, 3)
    # Interpolation
    t = f(grid)
    n00 = n000*(1-t[:,:,:,0]) + t[:,:,:,0]*n100
    n10 = n010*(1-t[:,:,:,0]) + t[:,:,:,0]*n110
    n01 = n001*(1-t[:,:,:,0]) + t[:,:,:,0]*n101
    n11 = n011*(1-t[:,:,:,0]) + t[:,:,:,0]*n111
    n0 = (1-t[:,:,:,1])*n00 + t[:,:,:,1]*n10
    n1 = (1-t[:,:,:,1])*n01 + t[:,:,:,1]*n11
    return ((1-t[:,:,:,2])*n0 + t[:,:,:,2]*n1)

def lerp(a,b,x):
    "linear interpolation"
    return a + x * (b-a)

def fade(t):
    "6t^5 - 15t^4 + 10t^3"
    return 6 * t**5 - 15 * t**4 + 10 * t**3

def gradient(h,x,y):
    "grad converts h to the right gradient vector and return the dot product with (x,y)"
    vectors = np.array([[0,1],[0,-1],[1,0],[-1,0]])
    g = vectors[h%4]
    return g[:,:,0] * x + g[:,:,1] * y


In [3]:
# read change rate, write change rate, consumption rate, self consumption rate
rcr = 0.3
wcr = 0.7
ccr = 0.2
sccr = 0.2

# memory unit pixel size
ps = 3
# normal spray pad size
pad = 2
# consumption spray pad size
consumption_pad = ps * 2

# blood spray color
BLOOD_SPRAY_COLOR = 1
BACKGROUND_COLOR = 0
FRAME_COLOR = 0
AGENT_COLOR = 0.5

pad_max = np.max([pad, consumption_pad])

In [4]:
# # Colors are dark and bright
# WARRIOR_COLORS = (((30,30,30), (200,200,200)),
#                   ((0,0,100), (0,0,255)),
#                   ((0,100,0), (0,255,0)),
#                 #   ((100,0,0), (255,0,0)),
#                   ((0,100,100), (0,255,255)),
#                   ((100,0,100), (255,0,255)),
#                   ((66,66,100), (170,170,255)),
#                   ((100,66,66), (255,170,170)),
#                   ((66,100,66), (170,255,170)),
#                   ((100,100,0), (255,255,0)),
#                   ((0,0,100), (0,0,255)),
#                   ((0,100,0), (0,255,0)),
#                 #   ((100,0,0), (255,0,0)),
#                   ((0,100,100), (0,255,255)),
#                   ((100,0,100), (255,0,255)),
#                   ((66,66,100), (170,170,255)),
#                   ((100,66,66), (255,170,170)),
#                   ((66,100,66), (170,255,170)),
#                   ((100,100,0), (255,255,0)))

# WARRIOR_COLORS = (((0,0,170), (200,200,200)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
#                   ((0,204,85), (0,0,255)),
 
#                   ((100,100,0), (255,255,0)))

# WARRIOR_COLORS = (((70,70,70), (125,125,125)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((5,5,5), (0,0,255)),
#                   ((100,100,0), (255,255,0)))

WARRIOR_COLORS = [[BACKGROUND_COLOR, 1]] + 13 * [[AGENT_COLOR, 1]]

# wc_arr = np.array(WARRIOR_COLORS, dtype=np.uint8)
wc_arr = np.array(WARRIOR_COLORS)

# specify the image size
shape = (10, 40, 20)

In [5]:
# reading the initial frame
i = 0
init_record = np.load("../games/test_"+str(i)+".npz")["record"]
# reshaping to the image size
rs_init_record = init_record.reshape((shape[0], shape[1], shape[2], 6))
# coloring based on the writer warrior
# reader warrior index: 4
# writer warrior index: 5
img = wc_arr[rs_init_record[:, :, :, 5], 0]

# increasing the image resolution
img = np.repeat(img, ps*ps*ps).reshape(shape[0]*ps, shape[1]*ps, shape[2]*ps)

# padding the image frame
img_arr = np.pad(np.array(img), ((pad_max, pad_max), (pad_max, pad_max), (pad_max, pad_max)), mode='constant', constant_values=[FRAME_COLOR])

In [6]:
def spray_read_3d(img_arr, record, pre_record, shape, ps, pad, pad_max, rcr, wc_arr):
    # read the new record
    record = np.load("../games/test_"+str(i)+".npz")["record"]
    
    # find the recent reads by agents
    recent_read = np.argwhere(record[:, 4] != pre_record[:, 4]).flatten()
    # extract the x, y indices
    x_ri, y_ri, z_ri = np.unravel_index(recent_read, shape)

    # generate the read spray kernels (randomly)
    read_spray_kernel = np.random.rand(
        recent_read.size, 
        ps + 2 * pad, 
        ps + 2 * pad, 
        ps + 2 * pad)
    # convert spray to indices
    rsk_index = np.argwhere(read_spray_kernel < rcr)

    
    # Right Side: reads the first element of rsk_index which is the id of the read change in this iteration, and with that index we read the id of the memory unit that has been changed from the recent_read, and with the memory unit id and 4 (read) we read the id of the agent that has executed the read, and we use this agent id to extract the color of the agent from wc_arr

    # Left Side: reads the first element of rsk_index which is the id of the read change in this iteration, and with that we read the x index that corresponds to the memory unit and multiply it with the pixel scale to find the top-left corner of the representing area in the high-resolution image, then we add the rsk_index column 1 to find the randomly selected pixel in the area that we need to change its color
    # print((x_ri[rsk_index[:, 0]] * ps + rsk_index[:, 1]).shape, (y_ri[rsk_index[:, 0]] * ps + rsk_index[:, 2]).shape)
    # print(img_arr[x_ri[rsk_index[:, 0]] * ps + rsk_index[:, 1], y_ri[rsk_index[:, 0]] * ps + rsk_index[:, 2]])
    img_arr[
        x_ri[rsk_index[:, 0]] * ps + pad_max - pad + rsk_index[:, 1], 
        y_ri[rsk_index[:, 0]] * ps + pad_max - pad + rsk_index[:, 2],
        z_ri[rsk_index[:, 0]] * ps + pad_max - pad + rsk_index[:, 3],
        ] = wc_arr[record[recent_read[rsk_index[:, 0]], 4], 0]

    return img_arr

def spray_write(img_arr, record, pre_record, shape, ps, pad, pad_max, wcr, wc_arr):
    # find the recent reads by agents
    recent_write = np.argwhere(record[:, 5] != pre_record[:, 5]).flatten()
    # extract the x, y indices
    x_wi, y_wi = np.unravel_index(recent_write, shape)

    # generate the read spray kernels (randomly)
    write_spray_kernel = np.random.rand(recent_write.size, ps, ps)
    # convert spray to indices
    wsk_index = np.argwhere(write_spray_kernel < wcr)

    img_arr[x_wi[wsk_index[:, 0]] * ps + pad_max - pad + wsk_index[:, 1], y_wi[wsk_index[:, 0]] * ps + pad_max - pad + wsk_index[:, 2]] = wc_arr[record[recent_write[wsk_index[:, 0]], 4], 0]
    
    return img_arr

def spray_self_consumption(img_arr, record, pre_record, shape, ps, pad, pad_max, sccr, wc_arr, bsc):
    # consumption condition are threefold:
    # 1. an action of read has executed at this memory unit
    # 2. the agent responsible for the read action is different from the agent that has lastly written in the same unit
    # 3. the agent responsible for the last write action is not 0 (which is the initialization action)
    recent_consumption = np.argwhere((record[:, 4] != pre_record[:, 4]) * (record[:, 4] == record[:, 5])* (record[:, 4] != 0)).flatten()
    # extract the x, y indices
    x_wi, y_wi = np.unravel_index(recent_consumption, shape)

    if recent_consumption.size > 0:
        # # generate the consumption spray kernels (randomly)
        # consumption_spray_kernel = np.random.rand(recent_consumption.size, ps + consumption_pad * 2, ps + consumption_pad * 2)
        # generate the consumption spray kernels (with perlin noise)
        noise_shape = (recent_consumption.size, ps + consumption_pad * 2, ps + consumption_pad * 2)
        consumption_spray_kernel = perlin(*np.meshgrid(
            np.linspace(0,2,noise_shape[0]* noise_shape[1],endpoint=False),
            np.linspace(0,2,noise_shape[2],endpoint=False))).reshape(noise_shape)
        
        consumption_spray_kernel -= consumption_spray_kernel.min() 
        consumption_spray_kernel /= consumption_spray_kernel.max()
        # all indices
        indices = np.argwhere(consumption_spray_kernel[0] + 0.0000001 >0)
        # distance of all cells to each other
        # # matrix of z indices
        # m = np.tile(indices, indices.shape[0]).reshape((indices.shape[0], indices.shape[0], 2))
        # # transpose of z indices
        # mt = m.transpose((1,0,2))
        # # computing the distance of all cells to each other
        # np.abs(m-mt).sum(axis=2)

        center_index = np.floor_divide(np.array(consumption_spray_kernel[0].shape), 2)
        dists = np.abs(indices - center_index).sum(axis=1)
        central_weights = dists / dists.max()

        # average of random kernel with distance to center
        consumption_spray_kernel = 0.7 * consumption_spray_kernel + 0.3 * central_weights.reshape(consumption_spray_kernel[0].shape) 
        # convert spray to indices
        csk_index = np.argwhere(consumption_spray_kernel < sccr)

        img_arr[x_wi[csk_index[:, 0]] * ps + pad_max - pad + csk_index[:, 1], y_wi[csk_index[:, 0]] * ps + pad_max - pad + csk_index[:, 2]] = np.array(bsc)
    
    return img_arr

def spray_consumption_3d(img_arr, record, pre_record, shape, ps, pad, pad_max, ccr, wc_arr, bsc):
    # consumption condition are threefold:
    # 1. an action of read has executed at this memory unit
    # 2. the agent responsible for the read action is different from the agent that has lastly written in the same unit
    # 3. the agent responsible for the last write action is not 0 (which is the initialization action)
    recent_consumption = np.argwhere((record[:, 4] != pre_record[:, 4]) * (record[:, 4] != record[:, 5]) * (record[:, 4] != 0)).flatten()
    # extract the x, y indices
    x_wi, y_wi, z_wi = np.unravel_index(recent_consumption, shape)
    
    if recent_consumption.size > 0:
        # # generate the consumption spray kernels (randomly)
        # consumption_spray_kernel = np.random.rand(recent_consumption.size, ps + consumption_pad * 2, ps + consumption_pad * 2)
        # generate the consumption spray kernels (with perlin noise)
        noise_shape = (recent_consumption.size, ps + consumption_pad * 2, ps + consumption_pad * 2, ps + consumption_pad * 2)
        
        consumption_spray_kernel = perlin_3d(
            (noise_shape[0]* noise_shape[1] * 1.0,noise_shape[2] * 1.0,noise_shape[3] * 1.0),
            (1,1,1)).reshape(noise_shape)
        # consumption_spray_kernel = perlin(*np.meshgrid(
        #     np.linspace(0,2,noise_shape[0]* noise_shape[1],endpoint=False),
        #     np.linspace(0,2,noise_shape[2],endpoint=False),
        #     np.linspace(0,2,noise_shape[3],endpoint=False)
        #     )).reshape(noise_shape)
        
        consumption_spray_kernel -= consumption_spray_kernel.min() 
        consumption_spray_kernel /= consumption_spray_kernel.max()
        # all indices
        indices = np.argwhere(consumption_spray_kernel[0] + 0.0000001 >0)

        center_index = np.floor_divide(np.array(consumption_spray_kernel[0].shape), 2)
        dists = np.abs(indices - center_index).sum(axis=1)
        central_weights = dists / dists.max()

        # average of random kernel with distance to center
        consumption_spray_kernel = 0.7 * consumption_spray_kernel + 0.3 * central_weights.reshape(consumption_spray_kernel[0].shape) 
        # convert spray to indices
        csk_index = np.argwhere(consumption_spray_kernel < ccr)

        img_arr[
            x_wi[csk_index[:, 0]] * ps + pad_max - pad + csk_index[:, 1], 
            y_wi[csk_index[:, 0]] * ps + pad_max - pad + csk_index[:, 2],
            z_wi[csk_index[:, 0]] * ps + pad_max - pad + csk_index[:, 3]
            ] = np.array(bsc)
    
    return img_arr


In [7]:
# initialize the prerecord
pre_record = init_record

# iterate over frames
for i in range(1, 5000):
    # read the new record
    record = np.load("../games/test_"+str(i)+".npz")["record"]
    
    # spray the read actions
    img_arr = spray_read_3d(img_arr, record, pre_record, shape, ps, pad, pad_max, rcr, wc_arr)

    # # spray the write actions
    # img_arr = spray_write(img_arr, record, pre_record, shape, ps, pad, pad_max, wcr, wc_arr)
    
    # spray the consumption
    img_arr = spray_consumption_3d(img_arr, record, pre_record, shape, ps, consumption_pad, pad_max, ccr, wc_arr, BLOOD_SPRAY_COLOR)

    # # spray the self consumption
    # img_arr = spray_self_consumption(img_arr, record, pre_record, shape, ps, consumption_pad, pad_max, ccr, wc_arr, BLOOD_SPRAY_COLOR)

    # update the prerecord
    pre_record = record

In [8]:
frame_path = "../out/I3/" + str(int(time.time())) + ".npz"
np.savez(frame_path, img_arr)

In [9]:
img_arr.shape

(42, 132, 72)

In [10]:
# new_img = Image.fromarray(img_arr)
# new_img.save("../out/I3/I3_" + str(time.time()) + ".png")
# new_img